In [38]:
from os import listdir
import numpy as np

In [39]:
my_path = 'Dataset/Classification-Train And Test/'
train_path = my_path + 'Train/'
test_path  = my_path + 'Test/'

In [40]:
def get_files_list(directory_path):
    files_path = []
    for class_path_name in listdir(directory_path):
        for data_file in listdir(directory_path + class_path_name):
            files_path.append(directory_path + class_path_name + '/' + data_file)
    return files_path

In [49]:
doc_train = get_files_list(train_path)
doc_test  = get_files_list(test_path)

In [50]:
doc_train[:10]

['Dataset/Classification-Train And Test/Train/Comp.graphics/data554.txt',
 'Dataset/Classification-Train And Test/Train/Comp.graphics/data565.txt',
 'Dataset/Classification-Train And Test/Train/Comp.graphics/data572.txt',
 'Dataset/Classification-Train And Test/Train/Comp.graphics/data574.txt',
 'Dataset/Classification-Train And Test/Train/Comp.graphics/data607.txt',
 'Dataset/Classification-Train And Test/Train/Comp.graphics/data608.txt',
 'Dataset/Classification-Train And Test/Train/Comp.graphics/data619.txt',
 'Dataset/Classification-Train And Test/Train/Comp.graphics/data624.txt',
 'Dataset/Classification-Train And Test/Train/rec.autos/data4025.txt',
 'Dataset/Classification-Train And Test/Train/rec.autos/data4032.txt']

In [51]:
doc_test[:10]

['Dataset/Classification-Train And Test/Test/Comp.graphics/data480.txt',
 'Dataset/Classification-Train And Test/Test/Comp.graphics/data488.txt',
 'Dataset/Classification-Train And Test/Test/rec.autos/data3982.txt',
 'Dataset/Classification-Train And Test/Test/rec.autos/data3992.txt',
 'Dataset/Classification-Train And Test/Test/rec.autos/data4011.txt',
 'Dataset/Classification-Train And Test/Test/sci.electronics/data6990.txt',
 'Dataset/Classification-Train And Test/Test/sci.electronics/data6993.txt',
 'Dataset/Classification-Train And Test/Test/sci.electronics/data7016.txt',
 'Dataset/Classification-Train And Test/Test/soc.religion.christian/data8750.txt',
 'Dataset/Classification-Train And Test/Test/soc.religion.christian/data8752.txt']